Загружаю видео и субтитры

In [1]:
!pip install git+https://github.com/ytdl-org/youtube-dl.git@master#egg=youtube_dl

  Cloning https://github.com/ytdl-org/youtube-dl.git (to revision master) to /tmp/pip-install-x00au1qc/youtube-dl_2acdd9cf617d49929c12662845528b40
  Running command git clone --filter=blob:none --quiet https://github.com/ytdl-org/youtube-dl.git /tmp/pip-install-x00au1qc/youtube-dl_2acdd9cf617d49929c12662845528b40
  Resolved https://github.com/ytdl-org/youtube-dl.git to commit be008e657d79832642e2158557c899249c9e31cd
  Preparing metadata (setup.py) ... done
  Created wheel for youtube_dl: filename=youtube_dl-2021.12.17-py2.py3-none-any.whl size=1944011 sha256=4f6f157ce5bbb622eef4c6c00555afd4cedd52b10aa4a86ee610f8664bde28c2
  Stored in directory: /tmp/pip-ephem-wheel-cache-a2zwi2k2/wheels/b8/03/62/9c414b89a26da510b0a6d984b0ba74200d591e3d0abfa72aa8
Successfully built youtube_dl


In [2]:
!youtube-dl --extract-audio --audio-format wav https://www.youtube.com/watch?v=stLw-2Ob8xQ&ab_channel=WIRED

[youtube] stLw-2Ob8xQ: Downloading webpage
[youtube] stLw-2Ob8xQ: Downloading player da154528
[dashsegments] Total fragments: 1
[download] Destination: Pedro Pascal & Oscar Isaac Answer the Web's Most Searched Questions _ WIRED-stLw-2Ob8xQ.m4a
[download] 100% of 7.70MiB in 00:00
[ffmpeg] Correcting container in "Pedro Pascal & Oscar Isaac Answer the Web's Most Searched Questions _ WIRED-stLw-2Ob8xQ.m4a"
[ffmpeg] Destination: Pedro Pascal & Oscar Isaac Answer the Web's Most Searched Questions _ WIRED-stLw-2Ob8xQ.wav
Deleting original file Pedro Pascal & Oscar Isaac Answer the Web's Most Searched Questions _ WIRED-stLw-2Ob8xQ.m4a (pass -k to keep)


In [3]:
!youtube-dl --write-auto-sub --convert-subs=srt --skip-download https://www.youtube.com/watch?v=stLw-2Ob8xQ&ab_channel=WIRED

[youtube] stLw-2Ob8xQ: Downloading webpage
[info] Writing video subtitles to: Pedro Pascal & Oscar Isaac Answer the Web's Most Searched Questions _ WIRED-stLw-2Ob8xQ.en.vtt


2.1 С помощью speech_recognition  https://pypi.org/project/SpeechRecognition/

In [12]:
import speech_recognition as sr

from os import path
AUDIO_FILE = "pedropascal2.wav"

r = sr.Recognizer()
with sr.AudioFile(AUDIO_FILE) as source:
    audio = r.record(source)

try:
    audio_transcribe_sr = r.recognize_sphinx(audio)
except sr.UnknownValueError:
    print("Sphinx could not understand audio")
except sr.RequestError as e:
    print("Sphinx error; {0}".format(e))

In [13]:
output_file = "transcription_speechrecognition.txt"
with open(output_file, "w") as file:
    file.write(audio_transcribe_sr)

2.2  Facebook wav2vec https://huggingface.co/facebook/wav2vec2-base-960h

In [1]:
from transformers import pipeline
transcriber = pipeline("automatic-speech-recognition", "facebook/wav2vec2-base-960h")
subs = transcriber("/content/pedropascal2.wav")

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You sho

In [4]:
output_file_2 = "transcription_facebook.txt"
with open(output_file_2, "w") as file:
    file.write(str(subs))

2.3 OpenAI whisper https://huggingface.co/openai/whisper-base.en

In [6]:
transcriber = pipeline("automatic-speech-recognition", "openai/whisper-tiny.en")
subs = transcriber("/content/pedropascal2.wav")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
output_file_2 = "transcription_openai.txt"
with open(output_file_2, "w") as file:
    file.write(str(subs))

Смотрю WER


In [14]:
with open('/content/goldenstandart.txt', 'r') as file:
    references = file.read()

In [15]:
references =  references.replace(".", "").replace(",", "").replace("'", "")
references = references.lower()
references = [references]

In [16]:
with open('/content/transcription_speechrecognition.txt', 'r') as file:
    predictions_sr = file.read()
    predictions_sr = [predictions_sr]

In [37]:
with open('/content/youtube_transcription.txt', 'r') as file:
    predictions_youtube = file.read()
    predictions_youtube = [predictions_youtube]

In [40]:
with open('/content/transcription_facebook.txt', 'r') as file:
    predictions_fb = file.read()
    predictions_fb = [predictions_fb]

In [19]:
with open('/content/transcription_openai.txt', 'r') as file:
    predictions_openai = file.read()
    predictions_openai = [predictions_openai]

WER speech_recognition

In [23]:
from evaluate import load
wer = load("wer")
wer_score_sr = wer.compute(predictions=predictions_sr, references=references)

In [24]:
wer_score_sr

0.9090909090909091

WER с youtube авто разметкой

In [38]:
wer2 = load("wer")
wer_score_youtube = wer2.compute(predictions=predictions_youtube, references=references)

In [39]:
wer_score_youtube

0.5833333333333334

WER c facebook

In [41]:
wer3 = load("wer")
wer_score_fb = wer3.compute(predictions=predictions_fb, references=references)

In [42]:
wer_score_fb

1.0

Как так? Не оч понятно

WER для OPENAI

In [34]:
wer4 = load("wer")
wer_score_openai = wer4.compute(predictions=predictions_openai, references=references)

In [35]:
wer_score_openai

0.9621212121212122